In [1]:
import scipy
import logging
import atlite
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [3]:
from init import *

In [5]:
from src.config import *

In [11]:
wind = xr.open_mfdataset([INTERIM_DIR / 'wind' / f'wind_2012-{month:02d}.nc' for month in MONTHS])